In [ ]:
import json
import random
from rich import print

## Load data

### Load country data generated by api notebook

In [ ]:
with open("saved_channels.json") as f:
    data = json.load(f)

In [ ]:
radio_countries = list(data.keys())

In [ ]:
len(radio_countries)

In [ ]:
random.sample(radio_countries, 4)

### Natural Earth

In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
ne = gpd.read_file("data/ne_50m_admin_0_countries.geojson")

In [ ]:
ne.shape

In [ ]:
len(ne["ADM0_TLC"].unique())

In [ ]:
len(ne["ISO_A3"].unique())

In [ ]:
ne[ne.duplicated(subset=['ISO_A3'])][["ADMIN", "ISO_A3", "SOV_A3", "ADM0_TLC"]]

In [ ]:
ne["MATCH"] = False

In [ ]:
def find(country):
    cols = ["NAME", "ADMIN", "NAME_LONG"]
    for col in cols:
        match_idxs = ne.index[ne[col].str.fullmatch(c)].tolist()
        if match_idxs:
            idx = match_idxs[0]
            ne.loc[idx, ("MATCH")] = True
            return idx
    return None

In [ ]:
no_match = []
match = []

for c in radio_countries:
    idx = find(c)
    if idx:
        match.append((c, idx))
    else:
        no_match.append(c)

In [ ]:
len(ne[ne["MATCH"] == True])

In [ ]:
len(match)

In [ ]:
len(no_match)

In [ ]:
len(radio_countries)

In [ ]:
pd.DataFrame(match)

In [ ]:
from thefuzz import fuzz
from thefuzz import process

In [ ]:
# ne_still = ne
ne_still = ne[ne["MATCH"] == False]

In [ ]:
def fuzzy_find(country, col):
    found = process.extract(
        country, ne_still[col].to_dict(), limit=3, scorer=fuzz.partial_ratio)
    for f in found:
        if f[1] > 80:
            return f
    return None

In [ ]:
now_found = []
still_no_match = []

for c in no_match:
    match = fuzzy_find(c, "ADMIN")
    if match:
        now_found.append((c, match[-1]) + (match[0], match[1]))
    else:
        still_no_match.append(c)

In [ ]:
pd.DataFrame(now_found, columns=["radio", "ne_idx", "ne_admin", "score"])

Everything good except:

- Iraq
- Guyana

In [ ]:
len(no_match)

In [ ]:
len(still_no_match)

In [ ]:
for c in no_match:
    match1 = fuzzy_find(c, "ADMIN")
    match2 = fuzzy_find(c, "NAME_LONG")
    
    if match1 or match2:
        print("-------")
        print(c)
    
    if match1:
        print("ADMIN", match1)
    if match2:
        print("NAME_LONG", match2)
    

In [ ]:
cols = ["NAME", "ADMIN", "NAME_LONG"]
ne_full = set(pd.concat([ne[col] for col in cols]))

In [ ]:
fuzzy = {}
for c in no_match:
    # found = process.extract(c, ne_full, limit=3, scorer=fuzz.partial_ratio)
    found = process.extract(c, ne_full, limit=3)
    fuzzy[c] = found

In [ ]:
for k, v in fuzzy.items():
    print(k, sorted(v, key=lambda x: x[1], reverse=True))

In [ ]:
matches = []

no_match_datamaps_countries = []

for c1 in datamaps_countries:
    match = False
    for c2 in radio_countries:
        if c1.lower() == c2.lower():
            matches.append((c1, c2))
            match = True
    if not match:
        no_match_datamaps_countries.append(c1)

### Load datamaps data

In [ ]:
with open("data/datamaps.json") as f:
    datamaps = json.load(f)

In [ ]:
datamaps_countries = [
    x["properties"]["name"]
    for x in datamaps
]

In [ ]:
len(datamaps_countries)

## Compare

In [ ]:
matches = []

no_match_datamaps_countries = []

for c1 in datamaps_countries:
    match = False
    for c2 in radio_countries:
        if c1.lower() == c2.lower():
            matches.append((c1, c2))
            match = True
    if not match:
        no_match_datamaps_countries.append(c1)

In [ ]:
len(matches)

In [ ]:
no_match_datamaps_countries

In [ ]:
matches = []

no_match_radio_countries = []

for c1 in radio_countries:
    match = False
    for c2 in datamaps_countries:
        if c1.lower() == c2.lower():
            matches.append((c1, c2))
            match = True
    if not match:
        no_match_radio_countries.append(c1)

In [ ]:
no_match_radio_countries